# Setup

In [1]:
import datetime, json, re, math, sys
import numpy as np
import numpy #lol
import matplotlib.pyplot as plt
from scipy.stats import linregress as lm
import scipy
import random
from collections import Counter, defaultdict
from __future__ import division
% matplotlib inline
#import the files
sys.path.append('../API')
import main

## Mu analysis

In [2]:
def preprocess(text):
    counts = Counter()
    text = text.encode("utf8")
    words = []
    for word in re.split(" ", text):
        if word not in counts:
            words.append(word)
        counts[word] += 1
        
    return counts

def order(counts):
    words = counts.keys()
    ps = np.array(counts.values())
    ps = ps/float(sum(ps))
    N = len(words)
    return np.random.choice(words, size=N, replace=False, p=ps)

def innovationrate(counts, reps = 2, termmax = 100):
    
    N = len(counts)    
    FN = sum(counts.values())

    ns = range(1,N+1)
    Mn = [0 for n in ns]
    
    for rep in range(reps):
        n = 0
        Fn = 0
        Msum = 0
        for n, word in zip(ns, order(counts)):

            f = counts[word]
            Fn += f

            if n == N:
                break 

            ## compute In and Jn
            In = Fn - (n - 1 + int(Msum))
            Jn = FN - (n - 1 + int(Msum))

            ## compute the average
            ms = np.array(range(1, min([In,termmax])+1))

            logfacts = np.log10(In - ms) - np.log10(Jn - ms)
            prods = 10**np.cumsum(logfacts)
            terms = ms*prods*(Jn - In)/(Jn - ms)
            termsum = sum(terms)
            Mn[n] += termsum/reps

            Msum += termsum

    return 1/(1 + np.array(Mn)), np.array(ns)

def decayExponent(text):
    counts = preprocess(text)
    reps = int(round(0.5 + (5000. / len(counts))))
    termmax = 1000
    alphas, ns = innovationrate(counts, reps, termmax)
    
    ix = range(int(len(ns)*1/3.),len(ns))

    x = np.log10(ns)[ix]
    y = np.log10(alphas)[ix]

    mu, b, r, p, err = lm(x,y)

    return -mu, sum(counts.values())

In [3]:
all_comments = {user : main.userText(user) for user in main.COMMENTS_BY_USER}

### The actual function

In [4]:
def getMu(userID, all_comments):
    comments = all_comments[userID]
    total_text = ""
    for comment in comments:
        total_text = total_text + comment + " "
    if total_text.strip(): #in case there's no text
        mu, numwords = decayExponent(total_text)
        return mu, numwords
    else:
        return 0.0, 0

## Number of links

In [5]:
def getNumLinks(userID, all_comments):
    numlinks = []
    comments = all_comments[userID]
    for comment in comments:
        links = re.findall(r'(https?://[^\s]+)', comment)
        numlink = len(links)
        numlinks.append(numlink)
    avg_numlinks = np.mean(numlinks)
    return avg_numlinks

## User's max daily comments

In [6]:
def maxDailyComments(UID):
    days = Counter()
    for comment in main.COMMENTS_BY_USER[UID]:
        date = comment['time'][0:10]
        days[date] += 1
    return(max(days.values()))

## Average response time

In [7]:
def avg_response(userID, choice = "All"):
    """get the avg response time for _userID"""
    comment_times = main.userResponse(userID, choice)
    if not comment_times:
        return None
    else:
        return np.mean(comment_times)

## Deviation from thread mean

In [8]:
def thread_deviation(userID, choice = "All"):
    """get the average deviation from the thread's mean response time for
    comments made by _userID_"""
    deviations = []
    thread_dict = defaultdict(list)
    threads = main.userThreads(userID, choice)
    comment_times = main.userResponse(userID, choice)
    if comment_times:
        for thread, comment_time in zip(threads, comment_times):
            #need to organize this by thread
            thread_dict[thread].append(comment_time)
        for threadID in thread_dict:
            thread_deviations = []
            thread_times = main.threadResponse(threadID, choice)
            thread_mean = np.mean(thread_times)
            for response in thread_dict[threadID]:
                thread_deviations.append(response - thread_mean)
            avg_thread_deviation = np.mean(thread_deviations)
            #now add to the total deviations
            deviations.append(avg_thread_deviation)
        return np.mean(deviations)
    else:
        return None

## Average comment length

In [9]:
def comment_length(userID, choice = "All"):
    """get the avg length of comments made by _userID_"""
    comments = main.userText(userID, choice)
    lengths = [len(comment) for comment in comments]
    return np.mean(lengths)

# Cross-Validation

### Classification function

In [10]:
def classify(userID, thresholds = [2086.68, 0, 0.2, 0.5, 0, 0.25, 100, 10]):
    #     delta_t = userFeatures[userID][0]
    #     mu = userFeatures[userID][2]
    #     l = userFeatures[userID][4]
    #     c_bar = userFeatures[userID][5]
    #     d_max = userFeatures[userID][6]
    # i think that threshold = [avg response time,
    #                           deviation from thread mean,
    #                           mu lower,
    #                           mu upper,
    #                           mu word count,
    #                           avg number links,
    #                           avg comment length,
    #                           max comments per day]
    # but not 100% sure???
    if userID in all_users:
        delta_t = all_users[userID][4]
        mu = all_users[userID][0]
        l = all_users[userID][2]
        c_bar = all_users[userID][6]
        d_max = all_users[userID][3]
        thread_avg = all_users[userID][5]
    else:
        delta_t = avg_response(userID)
        mu, _ = getMu(userID, all_comments)
        l = getNumLinks(userID, all_comments)
        c_bar = comment_length(userID)
        d_max = maxDailyComments(userID)
        thread_avg = thread_deviation(userID)
    
    if mu < thresholds[2] or mu > thresholds[3] or l > thresholds[5] or d_max > thresholds[7] or c_bar > thresholds[6]:
        if delta_t > thresholds[0] or thread_avg > thresholds[1]:
            return False
        else:
            return True
    else:
        return False
        
#     if mu > thresholds[2] and mu < thresholds[3]:
#         if l > thresholds[5]:
#             if delta_t > thresholds[0]:
#                 return False
#             else:
#                 return True
#         elif d_max > thresholds[7]:
#             if delta_t > thresholds[0]:
#                 return False
#             else:
#                 return True
#         elif c_bar > thresholds[6]:
#             if delta_t > thresholds[0]:
#                 return False
#             else:
#                 return True
#         else:
#             return False
#     elif delta_t > thresholds[0]:
#         return False
#     else:
#         return True

In [11]:
def split_users(list_users):
    """splits the list of userIDs _users_ into 10 equally-sized, random samples
    and returns a nested list representing them"""
    random.seed(0)
    random.shuffle(list_users)
    data_split = [[] for i in range(10)]
    for i, user in enumerate(list_users):
        batch_num = i % 10
        data_split[batch_num].append(user)
    return data_split

Create the function itself. We want to apply this to the 7 different parameters, so allow for one of the arguments
to be the parameter:
* 0 = average response time
* 1 = deviation from thread mean response time
* 2 = mu lower bound
* 3 = mu upper bound
* 4 = mu word count
* 5 = average number links
* 6 = avg comment length
* 7 = max comments/day

In [12]:
with open('ALL_USER_STATS.json', 'r') as f:
    all_users = json.load(f)
    
#get the bots info
with open('../../data/annotation.json', 'r') as f2:
    annotation = json.load(f2)
    
BOTS = [user for bucket in annotation for user in annotation[bucket] if annotation[bucket][user] != '1']

In [13]:
def crossValidateParam(parameter, guess_range, n = 100):
    """performs 10-fold cross validation on the parameter which is found by using the function
    _param_function_. _all_users_ is a dict of the annotation data.
    _guess_range_ is a tuple of two numbers, which is the range of values that
    we'll be scanning through for the parameter. _n_ is the size of the mesh for the interval
    _guess_range_. returns a list of the best parameter value for each of the 10 folds"""
    
    data_split = split_users(all_users.keys())
    result = [(None, None) for _ in range(10)] #list of tuples (parameter, F1 score) for each fold
    
    for i, fold in enumerate(data_split):
        best_param = 0 
        max_F1 = -1000   #stores the F1 score associated with best_param
        copy = list(data_split)   #we don't want to mess up the data
        test = copy.pop(i)   
        train = [user for fold in copy for user in fold]  #flatten the list
        #now start scanning the parameter values
        for scan_num in range(n):
            step_size = (guess_range[1] - guess_range[0]) / n  #increasing by this much each iteration
            param_value = guess_range[0] + step_size * scan_num  #test this paramater value
            param_F1 = cross_helper(train, parameter, param_value)
            #check if this is better than what we already have
            if param_F1 > max_F1:
                #update our values
                best_param = param_value
                max_F1 = param_F1
        
        #now that we have the best param for the fold, apply it to the test data
        fold_F1 = cross_helper(test, parameter, best_param)
        result[i] = (best_param, fold_F1)
    return result
        
def cross_helper(users, parameter, param_value):     
    """takes a list of users with a parameter and its value and returns the F1
    score obtained by using that value of the parameter in classification on only
    the users desired"""
    #TODO: make this give you the ability to input multiple parameters, not just one
    # so that we can use optimization in scipy or something
    thresholds = [float('Inf'), float('Inf'), -float('Inf'), float('Inf'),
                  0, float('Inf'), float('Inf'), float('Inf')]
    if parameter in [0, 1]:
        thresholds = [float('Inf'), float('Inf'), 0, -float('Inf'), 0, 0, 0, 0]  #just need one to set off the "OR" switch
    thresholds[parameter] = param_value
    return create_matrix(users, thresholds) 

def create_matrix(users, thresholds):
    """classifies each user in _users_ according to _thresholds_ and creates the confusion matrix. returns
    the corresponding F1 score"""
    confusion = Counter()
    for user in users:
        classification = classify(user, thresholds)
        if user in BOTS:  #the user is a bot
            if classification:
                confusion["tp"] += 1 #classify says bot, is bot
            else:
                confusion["fn"] += 1  #classify says human, is bot
            
        else: #the user is a human
            if classification:
                confusion["fp"] += 1  #classify says bot, but human
            else:
                confusion["tn"] += 1 #classify says human, is human
    return eval_F1(confusion)

def eval_precision(confusion_matrix):
    """given the Counter _confusion_matrix_, calculates and returns the
    precision"""
    precision = 0.0
    try:
        precision = confusion_matrix["tp"] / (confusion_matrix["tp"] + confusion_matrix["fp"])
    except:
        pass
    return precision

def eval_recall(confusion_matrix):
    """given the Counter _confusion_matrix_, calculates and returns the
    recall"""
    recall = 0.0
    try:
        recall = confusion_matrix["tp"] / (confusion_matrix["tp"] + confusion_matrix["fn"])
    except:
        pass
    return recall

def eval_F1(confusion_matrix):
    """calculates and returns the F1 score"""
    precision = eval_precision(confusion_matrix)
    recall = eval_recall(confusion_matrix)
    F1 = 0.0
    try:
        F1 = (2 * precision * recall / (precision + recall))
    except:
        pass
    return F1

def total_eval(confusion_matrix):
    """does all 3"""
    return eval_precision(confusion_matrix), eval_recall(confusion_matrix), eval_F1(confusion_matrix)
            

In [14]:
def run_cross():
    """runs the cross-validation process on each parameter in order, and makes a list of the optimal values for
    each. then applies these optimal values as thresholds to the full data set as a measure of the usefulness of
    our technique"""
    parameters = [0, 1, 5, 6, 7]  #only want to look at these 
    intervals = [(0, 5000), (-5000, 5000), (0, .1), (0, 300), (0, 100)]
    thresholds = [2000, 0, -float('Inf'), float('Inf'), 0, 0, 0, 0]
    for param, interval in zip(parameters, intervals):
        results = crossValidateParam(param, interval)
        temp_sum = 0
        for result in results:
            temp_sum += result[0]
        param_result = temp_sum / 10
        thresholds[param] = param_result  #update the parameter value
        
    
    #now run this on the full set
    confusion = Counter()
    for user in all_users.keys():
        classification = classify(user, thresholds)
        if user in BOTS:
            if classification:
                confusion["tp"] += 1 #classify says bot, is bot
            else:
                confusion["fn"] += 1  #classify says human, is bot

        else: #the user is a human
            if classification:
                confusion["fp"] += 1  #classify says bot, but human
            else:
                confusion["tn"] += 1 #classify says human, is human
    #display the optimal parameters
    print thresholds
    return eval_precision(confusion), eval_recall(confusion), eval_F1(confusion)

In [15]:
print crossValidateParam(1, (-5000,5000))

[(1300.0, 0.205607476635514), (-1700.0, 0.11764705882352942), (1300.0, 0.2522522522522523), (-1700.0, 0.2), (1300.0, 0.2727272727272727), (1300.0, 0.2522522522522523), (1300.0, 0.2037037037037037), (3000.0, 0.3157894736842105), (3000.0, 0.24778761061946902), (1000.0, 0.2727272727272727)]


* Without thread response deviation:
(0.41721854304635764, 0.8235294117647058, 0.553846153846154)

* With thread response deviation:
(0.41946308724832215, 0.8169934640522876, 0.5543237250554325)
* thresholds = [3235.0, 1010.0, -inf, inf, 0, 0.004200000000000001, 162.3, 30.0]
 

## TODO:
* histogram of deviation from mean thread response time, figure out how to use it (indicator or not)
* actually just do histograms for all the parameters
* then create a scatterplot matrix for all the parameters


## Scipy Optimization

In [18]:
def optimizeCross(thresholds = [3235.0, 1010.0, 0.004200000000000001, 162.3, 30.0]):
    """uses scipy.optimize to find best parameters for each fold of training data,
    then tests it on the test data. will return a list of the best parameters found
    by taking their averages over the 10 iterations of optimization"""
    data_split = split_users(all_users.keys())
    results = [[None] * 5 for _ in range(10)] #store the thresholds obtained
    fold_F1 = [None] * 10
    for i, fold in enumerate(data_split):
        #TODO: implement scipy.optimize.minimize here!
        copy = list(data_split)   #we don't want to mess up the data
        test = copy.pop(i)   
        train = [user for fold in copy for user in fold]  #flatten the list
        answer = scipy.optimize.minimize(log_create_matrix,
                                         thresholds,
                                         args = (train,),
                                         method = 'Nelder-Mead')
        #print answer.x
        results[i] = answer.x
        fold_F1[i] = optimize_matrix(results[i], test)
    final_params = [np.mean(param) for param in zip(*results)] #take the means across the folds
    
    confusion = Counter()
    for user in all_users.keys():
        classification = optimize_classify(user, final_params)
        if user in BOTS:  #the user is a bot
            if classification:
                confusion["tp"] += 1 #classify says bot, is bot
            else:
                confusion["fn"] += 1  #classify says human, is bot
            
        else: #the user is a human
            if classification:
                confusion["fp"] += 1  #classify says bot, but human
            else:
                confusion["tn"] += 1 #classify says human, is human
    print final_params
    print fold_F1
    return total_eval(confusion)
  

def log_create_matrix(thresholds, users):
    """since we're performing a minimization procedure we'll be converting the F1 scores
    returned by create_matrix to be the negative logs, to allow for better accuracy. this
    is the function we'll be running the process on. temp_thresholds must be the first
    input as per scipy's requirements, and will just be the variables we're trying to 
    optimize (the three mu values won'be used yet.)"""
    return -math.log(optimize_matrix(thresholds, users))

def optimize_matrix(thresholds, users):
    """the old one doesn't seem to work with the optimization function, since
    we're trying to add the mu values. this version of the function will be the
    same, but we won't require any setting of the mu values"""
    confusion = Counter()
    for user in users:
        classification = optimize_classify(user, thresholds)
        if user in BOTS:  #the user is a bot
            if classification:
                confusion["tp"] += 1 #classify says bot, is bot
            else:
                confusion["fn"] += 1  #classify says human, is bot
            
        else: #the user is a human
            if classification:
                confusion["fp"] += 1  #classify says bot, but human
            else:
                confusion["tn"] += 1 #classify says human, is human
    return eval_F1(confusion)

def optimize_classify(userID, thresholds):
    """the mu values are messing the optimization up. this is exactly the same as the normal
    classify function except the mu values are completely taken out, for now."""

    if userID in all_users:
        delta_t = all_users[userID][4]
        l = all_users[userID][2]
        c_bar = all_users[userID][6]
        d_max = all_users[userID][3]
        thread_avg = all_users[userID][5]
    else:
        delta_t = avg_response(userID)
        l = getNumLinks(userID, all_comments)
        c_bar = comment_length(userID)
        d_max = maxDailyComments(userID)
        thread_avg = thread_deviation(userID)
    
    if l > thresholds[2] or d_max > thresholds[4] or c_bar > thresholds[3]:
        if delta_t > thresholds[0] or thread_avg > thresholds[1]:
            return False
        else:
            return True
    else:
        return False
    
    
    

In [19]:
optimizeCross()

[3176.8488895278083, 989.54603895500804, 0.0041883628274073624, 174.42818973379582, 30.156292945920008]
[0.5000000000000001, 0.3783783783783784, 0.5909090909090909, 0.7037037037037038, 0.5714285714285714, 0.5641025641025641, 0.42553191489361697, 0.625, 0.6341463414634146, 0.48979591836734687]


(0.42955326460481097, 0.8169934640522876, 0.563063063063063)

### Results of optimization:
(recall, precision, F1)
* threshold from just parameter scan: (0.41946308724832215, 0.8169934640522876, 0.5543237250554325)
* after scipy.optimize: (0.42955326460481097, 0.8169934640522876, 0.563063063063063)

Looks like the optimization just increased precision a bit. Recall was exactly the same